In [19]:
import numpy,datetime,re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher

In [20]:
#READ DATA:
IMDB_DF = pd.read_csv("imdb.csv")
IMDB_DF_SAMPLE = IMDB_DF.sample(10).astype(str)
imdb_header_list = list(IMDB_DF.columns.values)

#imdb_header
imdb_header_array = np.array(imdb_header_list)

In [3]:
ROTTEN_TOMATOES_DF = pd.read_csv("rotten_tomatoes.csv")
ROTTEN_TOMATOES_DF_SAMPLE = ROTTEN_TOMATOES_DF.sample(10).astype(str)
rt_header_list = list(ROTTEN_TOMATOES_DF.columns.values)
#rt_header
rt_header_array = np.array(rt_header_list)

In [4]:
#creation array to add sort index of matching
# For matching
Pref=np.zeros((len(imdb_header_array),len(rt_header_array)))
#for Tomatoes
pref_tom=np.zeros((len(imdb_header_array),len(rt_header_array)),dtype=np.int8)
ipref_tom=np.zeros((len(imdb_header_array),len(rt_header_array)),dtype=np.int8)
#for IMdb
ipref_imdb=np.zeros((len(rt_header_array),len(imdb_header_array)),dtype=np.int8)
pref_imdb=np.zeros((len(rt_header_array),len(imdb_header_array)),dtype=np.int8)

In [5]:
def check_short_matching(COL1,COL2):
    rate = 0.0
    if str(COL1) == 'nan' or str(COL2) == 'nan' :
        return 0
    COL1 = str(COL1)
    COL2 = str(COL2)
    sw = (fuzz.SequenceMatcher(None,COL1, COL2).ratio())
    return sw #/(max(len(COL1),len(COL2))*1.0)

In [6]:
def check_match(st1,st2):
    str1 = np.array((st1.values).astype(str))
    str2 = np.array((st2.values).astype(str))
    ret = np.zeros((len(str1),len(str2)))
    for i in range(len(str1)):
        for j in range(len(str2)):
            ret[i,j] = check_short_matching(str1[i],str2[j])
    return np.average(ret)

In [7]:
index_i = 0
for i in IMDB_DF_SAMPLE:
    index_j = 0
    for j in ROTTEN_TOMATOES_DF_SAMPLE:
        table1 = IMDB_DF_SAMPLE[i]
        table2 = ROTTEN_TOMATOES_DF_SAMPLE[j]
        Pref[index_i,index_j] = 100 * (np.float(check_match(table1, table2)))
        index_j+=1
    index_i+=1

In [8]:
#assign the array indexs of Pref sorted by the actual values decending based on the lenght of rt_header_array (17)
for i in range(len(rt_header_array)):
    ipref_tom[:,i]=np.argsort(Pref[:,i])
    ipref_tom[:,i]=ipref_tom[::-1,i]
    pref_tom[:,i]=np.sort(Pref[:,i])
    pref_tom[:,i]=pref_tom[::-1,i]
#ipref_tom

In [9]:
#assign the array indexs of Pref sorted by the actual values decending based on the lenght of imdb_header_array (13)
for i in range(len(imdb_header_array)):
    #get indexs values of rows and store them in the rows as well
    ipref_imdb[:,i] = np.argsort(Pref[i,:])
    ipref_imdb[:,i] = ipref_imdb[::-1,i]
    pref_imdb[:,i] = np.sort(Pref[i,:])

In [10]:
#create list of preferred_rankings_tomatoes with header
preferred_rankings_tomatoes = {}
for i in range(len(rt_header_array)):
    preferred_rankings_tomatoes[i] = ipref_tom[:,i]


In [11]:
#create list of preferred_rankings_Imdb with header
preferred_rankings_imdb = {}
for i in range(len(imdb_header_array)):
    preferred_rankings_imdb[i] = ipref_imdb[:,i]

In [12]:
tentative_engagements = []
free_imdbs = []
corresponacy1 = []

In [13]:
#Imdb which still need to propose and get accepted successfully
#Keep track of the headers that "may" end up together
def init_free_imdbs():
    #corresponacy1 = list.clear()
    #'''Initialize the arrays of tomates and imdbs to represent that they're all initially free and not engaged'''
    for imdb in preferred_rankings_imdb.keys():
        free_imdbs.append(imdb)

In [14]:
def begin_matching(imdb):
    #Find the first free Tomaten available to a Imdb at any given time
    for tomate in preferred_rankings_imdb[imdb]:
        #Boolean for whether tomate is taken or not
        taken_match = [couple for couple in tentative_engagements if tomate == couple [:][1]]
        if (len(taken_match) == 0):
            #tentatively engage the man and woman
            tentative_engagements.append([imdb, tomate])
            free_imdbs.remove(imdb)
            break
        elif (len(taken_match) > 0):
            current_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(taken_match[0][0])
            potential_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(imdb)
            if (current_imdb < potential_imdb):
                ii=[]
            else: 
                #The new imdb is engaged
                free_imdbs.remove(imdb)
                #The old imdb is now single
                free_imdbs.append(taken_match[0][0])
                #Update the fiance of the tomaten (tentatively)
                taken_match[0][0] = imdb
                break

In [15]:
def stable_matching():
    '''Matching algorithm until stable match terminates'''
    while (len(free_imdbs) > 0):
        for imdb in free_imdbs:
            begin_matching(imdb)

In [16]:
def output():
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        if(matching > 10.0):
            corresponacy1.append(item)
            print(i ,'. CORRECT \t(',imdb_header_array[item[0]],rt_header_array[item[1]],')with % of matching',matching)
            i+=1
        else:
            print(i,'. Wrong \t',imdb_header_array[item[0]],rt_header_array[item[1]],'with % of matching',matching)
            i+=1
    found =len(corresponacy1) - 1
    print('A:Number of the discovered correspondences that are in G(withoutID)',found)
    print('B:Number of all the discovered correspondences',len(corresponacy1))
    print('precision(A/B):',found/len(corresponacy1)*100)
    print('recall(A/Number of all the actual correspondences = 10):',(found/10)*100)

In [17]:
def main():
    tentative_engagements.clear()
    free_imdbs.clear()
    corresponacy1.clear()
    init_free_imdbs()

    #print(free_imdbs)
    stable_matching()
    print(tentative_engagements)
    output()
main()

[[10, 14], [3, 3], [4, 4], [11, 16], [8, 11], [9, 6], [1, 1], [5, 5], [0, 13], [2, 2], [7, 10], [12, 15], [6, 7]]
0 . CORRECT 	( Genre Genre )with % of matching 27.3134589399
1 . CORRECT 	( ReleaseDate Release Date )with % of matching 25.1534010645
2 . CORRECT 	( Director Director )with % of matching 26.3681284492
3 . CORRECT 	( Url Description )with % of matching 15.0358731342
4 . CORRECT 	( RatingValue RatingValue )with % of matching 11.8194444444
5 . CORRECT 	( ContentRating Actors )with % of matching 12.3237593461
6 . CORRECT 	( Name Name )with % of matching 22.4894525489
7 . CORRECT 	( Creator Creator )with % of matching 18.8501780718
8 . CORRECT 	( Id ReviewCount )with % of matching 18.0087365647
9 . CORRECT 	( YearRange Year )with % of matching 23.5555555556
10 . CORRECT 	( Duration Duration )with % of matching 14.2485380117
11 . Wrong 	 Description Filming Locations with % of matching 4.92916392912
12 . Wrong 	 Cast Cast with % of matching 2.18349511299
A:Number of the discover